# RedCap - Python API Notebook

An API (Application-Program Interface) is just the software that acts as a translator between an program and another program/application/database. It allows you to access another application without dealing with its own source code. 

The API in this case allows communication between RedCap database and Python files. The API converts code written in python to something understandable in RedCap so there can be clear communication between the python user and the database. 

See the following:
http://redcap-tools.github.io/PyCap/quickstart/

## Module imports

In [200]:
from redcap import Project  #Need to have PyCap installed as a python package

import pandas as pd # pandas is a module that contains functions to manage DataFrames
import numpy as np # numpy is a module with numerical python functions

from ipywidgets import widgets, Layout, interact, interactive, interactive_output, Dropdown #GUI modules
from IPython.display import display  #GUI modules


# In Python some programmers will make packages which consist of a group of modules. 
# A module is a python file with all the functions made my the developers. 
# In this case "PyCap" is the Package, one of the modules is "redcap"
# In the redcap code there is a class called "Project" which consists of all the relevant functions for a single project

In [201]:
lbl_apiurl = widgets.Label(description = 'API URL:')
edt_apiurl = widgets.Text(value='https://redcap.uct.ac.za/api/')
btn_select = widgets.Button(description = 'Select file',button_style='success')
output_file = widgets.Output()

display(edt_apiurl, btn_select,output_file)


Text(value='https://redcap.uct.ac.za/api/')

Button(button_style='success', description='Select file', style=ButtonStyle())

Output()

In [188]:
#api_url = 'https://redcap.uct.ac.za/redcap_v9.5.36/API/project_api.php?pid=856' #defined on Redcap project
#api_url = 'https://redcap.uct.ac.za/redcap_v9.5.36/API/project_api.php?pid=856'
api_url = 'https://redcap.uct.ac.za/api/'
api_key = '8BB3ED0C8EFD52AB5932DB834841F05A'#defined on redcap project
project = Project(api_url, api_key)

## Patients Report

In [189]:

data = project.export_reports(report_id='8206',raw_or_label_headers='label', raw_or_label = 'label')
df_patients = pd.DataFrame(data)

## Surgery Report

In [190]:
data = project.export_reports(report_id='8207',raw_or_label_headers='label', raw_or_label = 'label')
df_surgery = pd.DataFrame(data)

In [191]:
df_surgery.head()
surg_serial = np.array(df_surgery.surg_serial)
surgery_patient_serial = []

for i in range(len(surg_serial)):
    raw = str(surg_serial[i])
    new =''
    for j in range(5):
        new = new+raw[j] 
    surgery_patient_serial.append(new)
    
df_surgery['patient_serial'] = surgery_patient_serial

df_surgery.head()

,id_number,redcap_repeat_instrument,redcap_repeat_instance,surg_serial,surg_mrn,surg_hospital,surg_date,surg_session,surg_exp_tissue,surg_age_calc,surg_procedure,surg_diagnosis_notes,surgeries_complete,patient_serial
0,1,Surgeries,1,P0001-1,139390850,Red Cross Children's Hospital,2018-02-06,Afternoon,Temporal cortex,,Temporal lobectomy,Left temporal lobe dysplasia and epilepsy,Incomplete,P0001
1,2,Surgeries,1,P0002-1,166229914,Red Cross Children's Hospital,2018-02-22,Morning,Frontal cortex,,Frontal lobectomy,Right frontal lobe tumour - suspected high gra...,Incomplete,P0002
2,3,Surgeries,1,P0003-1,155693385,Red Cross Children's Hospital,2018-05-10,Morning,Temporal cortex,,Temporal lobectomy,Left choroidal fissure arachnoid cyst and epil...,Incomplete,P0003
3,4,Surgeries,1,P0004-1,161640388,Red Cross Children's Hospital,2018-05-22,Morning,Frontal cortex,,Frontal lobectomy,Left perirolandic area refractory epilepsy.,Incomplete,P0004
4,5,Surgeries,1,P0005-1,109058859,Red Cross Children's Hospital,2018-07-12,Morning,Frontal cortex,,Frontal lobectomy,Left frontal parafalcine cortical dysplasia an...,Incomplete,P0005


### Check if all surgery patient_serial in patients dataframe

In [192]:
df_patients.head()
patients_patient_serial  = list(df_patients.patient_serial)

for i in range(len(surgery_patient_serial)):
    
    if (surgery_patient_serial[i] in patients_patient_serial[:]) == False:
        print(surgery_patient_serial[i] + ' not found in patients report')



## Resection report

In [193]:
data = project.export_reports(report_id='8208',raw_or_label_headers='label', raw_or_label = 'label')
df_resections = pd.DataFrame(data)
df_resections.head()


resection_serial = np.array(df_resections.resection_serial)
resection_surgery_serial = []

for i in range(len(resection_serial)):
    raw = str(resection_serial[i])
    new =''
    for j in range(7):
        new = new+raw[j] 
    resection_surgery_serial.append(new)
    
df_resections['surg_serial'] = resection_surgery_serial

df_resections.head()



,id_number,redcap_repeat_instrument,redcap_repeat_instance,resection_serial,resection_area,resection_hemi,resection_time,resection_transporttime,resection_photo_1,resection_notes,resections_complete,surg_serial
0,1,Resections,1,P0001-11,Temporal cortex,Left,18:00,30,,,Incomplete,P0001-1
1,2,Resections,1,P0002-11,Frontal cortex,Right,10:00,30,,,Incomplete,P0002-1
2,3,Resections,1,P0003-11,Temporal cortex,Left,10:00,30,,,Incomplete,P0003-1
3,4,Resections,1,P0004-11,Frontal cortex,Left,10:00,30,,No notes in lab book describing this case...,Incomplete,P0004-1
4,5,Resections,1,P0005-11,Frontal cortex,Left,10:00,30,,,Incomplete,P0005-1


### Adding image string to resections dataframe

In [194]:
path_str_arr =[]
for i in range(len(df_resections.resection_serial)):
    path_str = "C:\\"  +"\\Resection photos\\"+ list(df_resections.resection_serial)[i] + '.jpeg'
    path_str_arr.append(path_str)
    
df_resections['photos'] = path_str_arr

df_resections.head()

,id_number,redcap_repeat_instrument,redcap_repeat_instance,resection_serial,resection_area,resection_hemi,resection_time,resection_transporttime,resection_photo_1,resection_notes,resections_complete,surg_serial,photos
0,1,Resections,1,P0001-11,Temporal cortex,Left,18:00,30,,,Incomplete,P0001-1,C:\\Resection photos\P0001-11.jpeg
1,2,Resections,1,P0002-11,Frontal cortex,Right,10:00,30,,,Incomplete,P0002-1,C:\\Resection photos\P0002-11.jpeg
2,3,Resections,1,P0003-11,Temporal cortex,Left,10:00,30,,,Incomplete,P0003-1,C:\\Resection photos\P0003-11.jpeg
3,4,Resections,1,P0004-11,Frontal cortex,Left,10:00,30,,No notes in lab book describing this case...,Incomplete,P0004-1,C:\\Resection photos\P0004-11.jpeg
4,5,Resections,1,P0005-11,Frontal cortex,Left,10:00,30,,,Incomplete,P0005-1,C:\\Resection photos\P0005-11.jpeg


### Check to see if resection_surg_serial is in surgery_serial

In [195]:
surg_serial  = list(df_surgery.surg_serial)


for i in range(len(resection_surgery_serial)):
    
    if (resection_surgery_serial[i] in surg_serial[:]) == False:
        print(resection_surgery_serial[i] + ' not found in surgery report')



## Tissue blocks report

In [196]:
data = project.export_reports(report_id='8209',raw_or_label_headers='label', raw_or_label = 'label')
df_tissues = pd.DataFrame(data)
df_tissues.head()

block_serial = np.array(df_tissues.block_serial)
block_resection_serial = []

for i in range(len(block_serial)):
    raw = str(block_serial[i])
    new =''
    for j in range(8):
        new = new+raw[j] 
    block_resection_serial.append(new)
    
df_tissues['resection_serial'] = block_resection_serial


df_tissues.head()


,id_number,redcap_repeat_instrument,redcap_repeat_instance,block_serial,block_datetime,block_type,block_photo,block_comments,block_fixtime,block_fixcomments,...,block_atac_count,block_ataclibprep,block_atacseq,block_atac_comments,block_snrna_count,block_snrnaseq,block_snrna_comments,block_status,tissue_blocks_complete,resection_serial
0,1,Tissue blocks,1,P0001-1101,2018-02-06 18:30,Acute slicing,,,,,...,,,,,,,,Used,Incomplete,P0001-11
1,2,Tissue blocks,1,P0002-1101,2018-02-22 10:30,Acute slicing,,,,,...,,,,,,,,Used,Incomplete,P0002-11
2,2,Tissue blocks,2,P0002-1102,2018-02-22 10:30,Golgi impregnated,,block; fixed then golgi; Current storage solut...,,,...,,,,,,,,Bio-banked,Incomplete,P0002-11
3,3,Tissue blocks,1,P0003-1101,2018-05-10 10:30,Acute slicing,,,,,...,,,,,,,,Used,Incomplete,P0003-11
4,4,Tissue blocks,1,P0004-1101,2018-05-22 10:30,Acute slicing,,,,,...,,,,,,,,Used,Incomplete,P0004-11


### Check to see if block_resection_serial is in resection_serial

In [197]:
resection_serial  = list(df_resections.resection_serial)


for i in range(len(block_resection_serial)):
    
    if (block_resection_serial[i] in resection_serial[:]) == False:
        print(block_resection_serial[i] + ' not found in resection report')


### Add image string to tissues dataframe

In [198]:
path_str_arr =[]
for i in range(len(df_tissues.block_serial)):
    path_str = "C:\\"  +"\\Tissue photos\\"+ list(df_tissues.block_serial)[i] + '.jpeg'
    path_str_arr.append(path_str)
    
df_tissues['photos'] = path_str_arr

df_tissues.head()

,id_number,redcap_repeat_instrument,redcap_repeat_instance,block_serial,block_datetime,block_type,block_photo,block_comments,block_fixtime,block_fixcomments,...,block_ataclibprep,block_atacseq,block_atac_comments,block_snrna_count,block_snrnaseq,block_snrna_comments,block_status,tissue_blocks_complete,resection_serial,photos
0,1,Tissue blocks,1,P0001-1101,2018-02-06 18:30,Acute slicing,,,,,...,,,,,,,Used,Incomplete,P0001-11,C:\\Tissue photos\P0001-1101.jpeg
1,2,Tissue blocks,1,P0002-1101,2018-02-22 10:30,Acute slicing,,,,,...,,,,,,,Used,Incomplete,P0002-11,C:\\Tissue photos\P0002-1101.jpeg
2,2,Tissue blocks,2,P0002-1102,2018-02-22 10:30,Golgi impregnated,,block; fixed then golgi; Current storage solut...,,,...,,,,,,,Bio-banked,Incomplete,P0002-11,C:\\Tissue photos\P0002-1102.jpeg
3,3,Tissue blocks,1,P0003-1101,2018-05-10 10:30,Acute slicing,,,,,...,,,,,,,Used,Incomplete,P0003-11,C:\\Tissue photos\P0003-1101.jpeg
4,4,Tissue blocks,1,P0004-1101,2018-05-22 10:30,Acute slicing,,,,,...,,,,,,,Used,Incomplete,P0004-11,C:\\Tissue photos\P0004-1101.jpeg


In [205]:
#df_tissues.to_csv('')
df_tissues.to_excel('C://Tissues.xlsx')